In [1]:
import k3d

origins = [1.0, 1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
vectors = [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]
colors = [0xff0000, 0x0000ff, 0x0000ff, 0xff0000, 0x0000ff, 0xff0000]


plot = k3d.plot()
vectors = k3d.vectors(origins, vectors, colors=colors, labels=['aa', 'bb', 'cc'], label_size=1.5)
vectors.head_size = 1.0

plot += vectors
plot.display()

Output()

In [23]:
import k3d
from math import sin, cos, pi

width = height = length = 20
model_matrix = [
    7.0,  5.0, -5.0, 0.0,
    0.0,  7.0,  7.0, 5.0,
    7.0, -5.0,  5.0, 0.0,
    0.0,  0.0,  0.0, 1.0
]

def f(x, y, z):
    return sin(float(x) / width * pi * 2.0), cos(float(y) / height * pi * 2.0), sin(float(z) / length * pi * 2.0)

colors = (0xFF0000, 0x00FF00) * width * height * length
vectors = [[[f(x, y, z) for x in range(width)] for y in range(height)] for z in range(length)]


plot = k3d.plot()
plot += k3d.vector_field(vectors, colors, model_matrix=model_matrix, use_head=True)
plot.display()
plot.objects[0].scale = 1.0
# plot.objects[0].use_head = False

[[[[ 0.          1.          0.        ]
   [ 0.30901699  1.          0.        ]
   [ 0.58778525  1.          0.        ]
   ...
   [-0.80901699  1.          0.        ]
   [-0.58778525  1.          0.        ]
   [-0.30901699  1.          0.        ]]

  [[ 0.          0.95105652  0.        ]
   [ 0.30901699  0.95105652  0.        ]
   [ 0.58778525  0.95105652  0.        ]
   ...
   [-0.80901699  0.95105652  0.        ]
   [-0.58778525  0.95105652  0.        ]
   [-0.30901699  0.95105652  0.        ]]

  [[ 0.          0.80901699  0.        ]
   [ 0.30901699  0.80901699  0.        ]
   [ 0.58778525  0.80901699  0.        ]
   ...
   [-0.80901699  0.80901699  0.        ]
   [-0.58778525  0.80901699  0.        ]
   [-0.30901699  0.80901699  0.        ]]

  ...

  [[ 0.          0.58778525  0.        ]
   [ 0.30901699  0.58778525  0.        ]
   [ 0.58778525  0.58778525  0.        ]
   ...
   [-0.80901699  0.58778525  0.        ]
   [-0.58778525  0.58778525  0.        ]
   [-0.30901699 

Output()

In [11]:
import numpy as np 

fig = k3d.plot()
n = 20
x = np.linspace(-5, 5, n)
y = np.linspace(-5, 5, n)
z = np.linspace(-5, 5, n)
xx, yy, zz = np.meshgrid(x, y, z)
uu, vv, ww = zz, yy, xx
xx, yy, zz, uu, vv, ww = [t.flatten().astype(np.float32) for t in [xx, yy, zz, uu, vv, ww]]
scale = 0.5
magnitude = np.sqrt(uu**2 + vv**2 + ww**2)
vectors = np.array((uu, vv, ww)).T * scale
origins = np.array((xx, yy, zz)).T
colors = k3d.helpers.map_colors(magnitude, k3d.matplotlib_color_maps.Plasma, [])
vec_colors = np.zeros(2 * len(colors))
for i, c in enumerate(colors):
    vec_colors[2 * i] = c
    vec_colors[2 * i + 1] = c
vec_colors = vec_colors.astype(np.uint32)
vec = k3d.vectors(
    origins = origins - vectors / 2,
    vectors = vectors,
    colors = vec_colors,
)
fig += vec
fig.display()

Output()

In [5]:
import os
import yt
import numpy as np
import k3d

from utils import *

DEBUG = True
hdf5_prefix = 'sn34_smd132_bx5_pe300_hdf5_plt_cnt_0'

def get_velocity_data(obj, x_range, y_range, z_range):
    """
    Retrieve velx, vely, velz, density, and temperature in a single function.
    """
    velx = obj["flash", "velx"][x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]].to('km/s').value
    vely = obj["flash", "vely"][x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]].to('km/s').value
    velz = obj["flash", "velz"][x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]].to('km/s').value
    dens = obj["flash", "dens"][x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]].to('g/cm**3').value
    temp = obj["flash", "temp"][x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]].to('K').value

    dz = obj['flash', 'dz'][x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]].to('cm').value
    mp = yt.physical_constants.mp.value  # Proton mass
    coldens = dens * dz / (1.4 * mp)

    print(f"velx.shape: {velx.shape}, vely.shape: {vely.shape}, velz.shape: {velz.shape}")
    return velx, vely, velz, coldens, temp


def visualize_velocity_field(velx, vely, velz, mask_cube, converted_points):
    """
    Visualize velocity field using k3d vectors.
    """
    # Masking the velocity components
    velx_masked = np.where(mask_cube, velx, np.nan)
    vely_masked = np.where(mask_cube, vely, np.nan)
    velz_masked = np.where(mask_cube, velz, np.nan)

    # Flatten and create 3D grid
    indices = np.argwhere(~np.isnan(velx_masked))
    origins = indices.astype(np.float32)

    if DEBUG:
        print(f"Original indices: {indices.shape}")

    # Randomly reduce the number of vectors to half
    if len(indices) > 1:
        selected_indices = np.random.choice(len(indices), size=len(indices) // 40, replace=False)
        indices = indices[selected_indices]
        origins = origins[selected_indices]

    vectors = np.array([velx_masked[indices[:, 0], indices[:, 1], indices[:, 2]],
                        vely_masked[indices[:, 0], indices[:, 1], indices[:, 2]],
                        velz_masked[indices[:, 0], indices[:, 1], indices[:, 2]]]).T

    # Normalize vectors to the range [-1, 1]
    max_vals = np.abs(vectors).max(axis=0)
    vectors = vectors / max_vals

    if DEBUG:
        print(f"New vectors: {vectors.shape}")
        print(f"Normalized vectors[0]: {vectors[0]}")

    scale = 20.0  # Adjust as needed
    magnitude = np.linalg.norm(vectors, axis=1)
    colors = k3d.helpers.map_colors(magnitude, k3d.matplotlib_color_maps.RdBu, [])
    vec_colors = np.zeros(2 * len(colors))
    for i, c in enumerate(colors):
        vec_colors[2 * i] = c
        vec_colors[2 * i + 1] = c
    vec_colors = vec_colors.astype(np.uint32)

    fig = k3d.plot()
    vec = k3d.vectors(
        origins=origins - vectors / 2,
        vectors=vectors * scale,
        colors=vec_colors,
        use_head=True
    )
    fig += vec

    # Add explosion points
    SB_center = k3d.points(positions=np.array(converted_points, dtype=np.float32),
                           point_size=3.0,
                           shader='3d',
                           opacity=1.0,
                           color=0xc30010)
    fig += SB_center

    fig.display()

    


# def segment_cube_roi(temp_cube, mask_cube, temp_thresh, lower_bound, upper_bound):
#     for current_z in range(upper_bound - lower_bound):
#         temp_slice = temp_cube[:, :, current_z + lower_bound]
#         temp_mask = temp_slice > np.power(10, temp_thresh)
#         mask_cube[:, :, current_z] = temp_mask

#     return mask_cube
def segment_cube_roi(temp_cube, dens_cube, mask_cube, temp_thresh, lower_bound, upper_bound):
    for current_z in range(upper_bound - lower_bound):
        dens_slice = normalize4thresholding(dens_cube[:, :, current_z + lower_bound])
        
        # threshold + connected component
        binary_mask = apply_otsus_thresholding(dens_slice)
        _, labels, _, _ = cv2.connectedComponentsWithStats(binary_mask, connectivity=8)

        # retrieve all mask only
        i = 0
        binary_mask = labels == i
        binary_mask = ~binary_mask

        # Apply temperature filter
        temp_slice = temp_cube[:, :, current_z + lower_bound]
        temp_mask = temp_slice > np.power(10, temp_thresh)
        binary_mask |= temp_mask

        mask_cube[:, :, current_z] = binary_mask
        # mask_cube[:, :, current_z] = temp_mask

    return mask_cube

def update_pos_pix256(filtered_data):
    converted_points = list(zip(
        pc2pix_256(filtered_data['posx_pc']) + 128,
        pc2pix_256(filtered_data['posy_pc']) + 128,
        pc2pix_256(filtered_data['posz_pc']) + 128
    ))

    return converted_points


In [6]:
start_timestamp = 380
end_timestamp =  380
incr = 10
hdf5_root = "/srv/data/stratbox_simulations/stratbox_particle_runs/bx5/smd132/sn34/pe300/4pc_resume/4pc"
lower_bound = 0
upper_bound = 256
pixel_boundary = 256
dat_filename = "SNfeedback.dat"
html_root = "/home/joy0921/Desktop/Dataset/MHD-3DIS/htmls"
temp_thresh = 5.5

for timestamp in range(start_timestamp, end_timestamp + 1, incr):
    time_Myr = timestamp2Myr(timestamp)
    ds = yt.load(os.path.join(hdf5_root, '{}{}'.format(hdf5_prefix, timestamp)))

    center = [0, 0, 0] * yt.units.pc
    arb_center = ds.arr(center, 'code_length')
    left_edge = arb_center + ds.quan(-500, 'pc')
    right_edge = arb_center + ds.quan(500, 'pc')
    obj = ds.arbitrary_grid(left_edge, right_edge, dims=(pixel_boundary,) * 3)

    x_range_scaled = (lower_bound, upper_bound)
    y_range_scaled = (lower_bound, upper_bound)
    z_range_scaled = (lower_bound, upper_bound)

    velx, vely, velz, dens_cube, temp_cube = get_velocity_data(obj, x_range_scaled, y_range_scaled, z_range_scaled)

    all_data = read_SNfeedback(hdf5_root=hdf5_root, filename=dat_filename)
    filtered_data = filter_data(all_data, range_coord=(-500, -500, 1000, 1000, -500, 500))
    converted_points = update_pos_pix256(filtered_data)

    if(DEBUG):
        print("Segmenting warm temperature bubbles...")
    mask_cube = np.zeros((pixel_boundary, pixel_boundary, upper_bound - lower_bound), dtype=bool)
    # mask_cube = segment_cube_roi(temp_cube, mask_cube, temp_thresh, lower_bound, upper_bound)
    mask_cube = segment_cube_roi(temp_cube, dens_cube, mask_cube, temp_thresh, lower_bound, upper_bound)


    visualize_velocity_field(velx, vely, velz, mask_cube, converted_points)




yt : [INFO     ] 2025-01-14 16:48:50,094 Particle file found: sn34_smd132_bx5_pe300_hdf5_part_0380
yt : [INFO     ] 2025-01-14 16:48:50,172 Parameters: current_time              = 6595634845883194.0
yt : [INFO     ] 2025-01-14 16:48:50,173 Parameters: domain_dimensions         = [  32   32 1280]
yt : [INFO     ] 2025-01-14 16:48:50,174 Parameters: domain_left_edge          = [-1.54283879e+21 -1.54283879e+21 -6.17135516e+22]
yt : [INFO     ] 2025-01-14 16:48:50,175 Parameters: domain_right_edge         = [1.54283879e+21 1.54283879e+21 6.17135516e+22]
yt : [INFO     ] 2025-01-14 16:48:50,176 Parameters: cosmological_simulation   = 0


velx.shape: (256, 256, 256), vely.shape: (256, 256, 256), velz.shape: (256, 256, 256)
Segmenting warm temperature bubbles...
Original indices: (6014148, 3)
New vectors: (150353, 3)
Normalized vectors[0]: [ 0.02361589 -0.01186331 -0.00087386]


/UBC-O/ashill01/anaconda3/envs/joy_env/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()